In [16]:
import pyxdf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ydata_profiling import ProfileReport

# Load Data

Load the data from the experimental input files for a single participant.

In [17]:
streams, header = pyxdf.load_xdf('./Path_Data/ID_100.xdf')
# for strm in streams:
#     print(f"Found stream {strm['info']['name'][0]} ({strm['info']['type'][0]}) with data shape "
#           f"{np.array(strm['time_series']).shape}")

stream_names = [strm['info']['name'][0] for strm in streams]
stream_types = [strm['info']['type'][0] for strm in streams]

In [68]:
data = {}

for stream in streams:
    stream_name = stream['info']['name'][0]
    stream_channels = {}
    for i, channels in enumerate(stream['info']['desc'][0]['channels'][0]['channel']):
        stream_channels[channels['label'][0]] = i
    stream_data = stream['time_series']
    #print(stream_name)
    #print(stream_channels)
    #print(stream_data)
    
    #example of data layout
    #stream_name: LuminanceStream
    #stream_channels: {'UserID': 0, 'BlockID': 1, 'ModelID': 2, 'MethodID': 3, 'Luminance': 4}
    #stream_data: [[1.0000000e+02 0.0000000e+00 9.9000000e+01 9.9000000e+01 1.1425743e-02][1.0000000e+02 0.0000000e+00 9.9000000e+01 9.9000000e+01 1.1425743e-02][1.0000000e+02 0.0000000e+00 9.9000000e+01 9.9000000e+01 1.1425743e-02]
    data_dict= {}
    for key in stream_channels.keys():
        data_dict[key] = np.array(stream_data)[:, stream_channels[key]]
    time_stamps = np.array(stream['time_stamps'])
    for i in range(len(time_stamps)):
        time_stamps[i] = round(time_stamps[i], 4)
    data_dict['time'] = time_stamps
    data[stream_name] = data_dict



In [69]:
dfs = {}
for stream_name, stream_data in data.items():
    dfs[stream_name] = pd.DataFrame(stream_data)

In [70]:
gaze_time = np.array(dfs['GazeStream']['time'])
luminance_time = np.array(dfs['LuminanceStream']['time'])

#loop through gaze time and round to 3 decimal places
for i in range(len(gaze_time)):
    gaze_time[i] = round(gaze_time[i], 4)

#loop through luminance time and round to 3 decimal places
# for i in range(len(luminance_time)):
#     luminance_time[i] = round(luminance_time[i], 4)

gaze_luminance_time = np.intersect1d(luminance_time, gaze_time)

gaze = dfs['GazeStream']
luminance = dfs['LuminanceStream']

#filter out data that is not in gaze_luminance_time
gaze = gaze[gaze['time'].isin(gaze_luminance_time)]


gaze.info()
luminance.info()

<class 'pandas.core.frame.DataFrame'>
Index: 132161 entries, 3 to 132386
Data columns (total 18 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   UserID                      132161 non-null  float32
 1   BlockID                     132161 non-null  float32
 2   ModelID                     132161 non-null  float32
 3   MethodID                    132161 non-null  float32
 4   ConvergenceDistance         132161 non-null  float32
 5   ConvergenceDistanceIsValid  132161 non-null  float32
 6   GazeOriginX                 132161 non-null  float32
 7   GazeOriginY                 132161 non-null  float32
 8   GazeOriginZ                 132161 non-null  float32
 9   GazeDirectionNormalizedX    132161 non-null  float32
 10  GazeDirectionNormalizedY    132161 non-null  float32
 11  GazeDirectionNormalizedZ    132161 non-null  float32
 12  GazeRayIsValid              132161 non-null  float32
 13  LeftEyeIsBlinking  